In [8]:
import requests
import json
import pandas as pd

In [2]:
api_key = 'AIzaS*******'

In [3]:
address="Tel Aviv , Israel"
url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address,api_key)
response = requests.get(url).json()  # if the response is of json format the .json() will load the json into a python object
print(type(response))

<class 'dict'>


In [25]:
response

<Response [200]>

In [11]:
with open('dests.txt', 'r') as file:
    destinations = file.read().splitlines()

In [18]:
 # Initialize an empty DataFrame   
data = {'Target': [], 'Distance_km': [], 'Duration (hour+minutes)': [], 'Longitude': [], 'Latitude': []}
df = pd.DataFrame(data)

# Process each destination
for destination in destinations:
    # Retrieve distance and duration using Distance Matrix API
    distance_url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins=Tel+Aviv&destinations={destination}&key={api_key}"
    distance_response = requests.get(distance_url)
    distance_data = distance_response.json()

    if distance_data['status'] == 'OK':
        distance = distance_data['rows'][0]['elements'][0]['distance']['text']
        duration = distance_data['rows'][0]['elements'][0]['duration']['text']
    else:
        distance, duration = None, None

    # Retrieve geolocation using Geocoding API
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={destination}&key={api_key}"
    geocode_response = requests.get(geocode_url)
    geocode_data = geocode_response.json()

    if geocode_data['status'] == 'OK':
        latitude = geocode_data['results'][0]['geometry']['location']['lat']
        longitude = geocode_data['results'][0]['geometry']['location']['lng']
    else:
        latitude, longitude = None, None

    # Add the data to the DataFrame
    df = df.append({
        'Target': destination,
        'Distance_km': distance,
        'Duration (hour+minutes)': duration,
        'Longitude': longitude,
        'Latitude': latitude
    }, ignore_index=True)

# Print the DataFrame
df

C:\Users\edenm\AppData\Local\Temp\ipykernel_11068\293447503.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\edenm\AppData\Local\Temp\ipykernel_11068\293447503.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\edenm\AppData\Local\Temp\ipykernel_11068\293447503.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\edenm\AppData\Local\Temp\ipykernel_11068\293447503.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\edenm\AppData\Local\Temp\ipykernel_11068\293447503.py:30: FutureWarning: The frame.append method is deprecated and 

,Target,Distance_km,Duration (hour+minutes),Longitude,Latitude
0,Istanbul,"1,815 km",21 hours 3 mins,28.978359,41.008238
1,Amsterdam,"4,533 km",2 days 0 hours,4.904139,52.367573
2,Valletta,"3,793 km",2 days 3 hours,14.514100,35.899237
3,Basel,"4,093 km",1 day 20 hours,7.588576,47.559599
4,Doha,"2,164 km",22 hours 39 mins,51.531040,25.285447


In [24]:
df_sorted = df.sort_values(by='Distance_km', ascending=False)
furthest_cities = df_sorted.head(3)
furthest_cities

,Target,Distance_km,Duration (hour+minutes),Longitude,Latitude
1,Amsterdam,"4,533 km",2 days 0 hours,4.904139,52.367573
3,Basel,"4,093 km",1 day 20 hours,7.588576,47.559599
2,Valletta,"3,793 km",2 days 3 hours,14.514100,35.899237
